Published on March 21, 2023. By Marília Prata, mpwolke

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://deepchecks.com/wp-content/uploads/2022/11/deepchecks-social.png)deepchecks.com

In [ ]:
! pip install deepchecks -U --user

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

import umap

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import math
from copy import deepcopy
from functools import partial
from itertools import combinations
from collections import Counter
from scipy.stats import chi2
# Import sklearn classes for model selection, cross validation, and performance evaluation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import seaborn as sns
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder

# Import libraries for Hypertuning
import optuna

# Import libraries for gradient boosting
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from catboost import Pool
from catboost.utils import get_roc_curve

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

rc = {
    "axes.facecolor": "#FFFEF8",
    "figure.facecolor": "#FFFEF8",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4
}
sns.set(rc=rc)
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']
from deepchecks.tabular.checks import FeatureDrift

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

original_df = pd.read_csv('/kaggle/input/predict-concrete-strength/ConcreteStrengthData.csv')
train_df = pd.read_csv('/kaggle/input/playground-series-s3e9/train.csv', index_col=[0])
test_df = pd.read_csv('/kaggle/input/playground-series-s3e9/test.csv', index_col=[0])
sample_sub = pd.read_csv('/kaggle/input/playground-series-s3e9/sample_submission.csv', index_col=[0])

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

plt.figure(figsize=(12, 10), dpi=80)
sns.heatmap(train_df.corr(), cmap="YlGnBu", annot=True)
plt.title('Correlation heatmap')
plt.show()

#To work with Deepchecks we must have train/test

In [ ]:
FeatureDrift().run(train_df.drop('Strength', axis=1), test_df)

#Original Dataset has 9 columns and train_df has 10 columns

Error:train and test requires to share the same features columns

In [ ]:
FeatureDrift().run(train_df.drop('Strength', axis=1), original_df.drop('Strength', axis=1))

In [ ]:
# kudos to @jcaliz for the visualization 
features = test_df.columns
n_bins = 50
histplot_hyperparams = {
    'kde':True,
    'alpha':0.4,
    'stat':'percent',
    'bins':n_bins
}

columns = features
n_cols = 3
n_rows = math.ceil(len(columns)/n_cols)
fig, ax = plt.subplots(n_rows, n_cols, figsize=(20, n_rows*4))
ax = ax.flatten()

for i, column in enumerate(columns):
    plot_axes = [ax[i]]
    sns.kdeplot(
        train_df[column], label='Train',
        ax=ax[i], color=palette[0]
    )
    
    sns.kdeplot(
        test_df[column], label='Test',
        ax=ax[i], color=palette[1]
    )
    
    sns.kdeplot(
        original_df[column], label='Original',
        ax=ax[i], color=palette[-4]
    )
    # titles
    ax[i].set_title(f'{column} Distribution');
    ax[i].set_xlabel(None)
    
    # remove axes to show only one at the end
    plot_axes = [ax[i]]
    handles = []
    labels = []
    for plot_ax in plot_axes:
        handles += plot_ax.get_legend_handles_labels()[0]
        labels += plot_ax.get_legend_handles_labels()[1]
        plot_ax.legend().remove()
    
for i in range(i+1, len(ax)):
    ax[i].axis('off')
    
fig.suptitle(f'Dataset Feature Distributions\n\n\n', ha='center',  fontweight='bold', fontsize=25)
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.95), fontsize=18, ncol=3)
plt.tight_layout()

#No clue why CementComponent returned an error above.

#Mahalanobis Distance to detect Outliers

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

def MahalanobisDist(data):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            vars_mean = []
            for i in range(data.shape[0]):
                vars_mean.append(list(data.mean(axis=0)))
            diff = data - vars_mean
            md = []
            for i in range(len(diff)):
                md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))

            return md
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!")

def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False

#Data manifold representation

Legend: -1 correspond to train dataset, 1 to original

Since it requires the original dataset you must add concrete strength Prediction by Mayur

In [ ]:
original_df['dataset_label'] = 'original'
train_df['dataset_label'] = 'train'
aux_target = 'dataset_label'

In [ ]:
import umap.umap_ as umap
import umap.plot

In [ ]:
aux_cols=['CementComponent','BlastFurnaceSlag','FlyAshComponent', 'WaterComponent',
'SuperplasticizerComponent', 'CoarseAggregateComponent', 'FineAggregateComponent', 'AgeInDays']

#It was suppose to be dataset_label. Since I got an error I changed to CementComponent

Just to make the UMAP below. And above I got so many times with CementComponent not in the index.

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

aux_df=full_df[aux_cols].iloc[::10, :]
mapper = umap.UMAP(n_neighbors=5, min_dist=0.3).fit(aux_df.drop('CementComponent', axis=1).values)
codes = {'original':1, 'train':-1}

#UMAP

In [ ]:
#By Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean

umap.plot.points(mapper, aux_df['CementComponent'].map(codes), background='black', theme='fire')
plt.show()

#Acknowledgements:

Alessandro Togni  https://www.kaggle.com/code/atogni85/pgs03e11-eda-data-drift-discrete-structure/notebook#EDA-and-data-clean